In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tifffile as tf
from scipy.ndimage.filters import gaussian_filter1d as gaussian
from glob import glob
from scipy.signal import savgol_filter, medfilt
import matplotlib.ticker as ticker
from scipy.optimize import curve_fit

In [2]:
# 1. load data into pandas data frame

# 2. convert data from I(t) to c_i(t) via equation for G quadruplex binding
    # - find t0
    # - 
# 3. calculate dc_i/dt for each GUV

# 4. calculate background intensity and convert this to c_0(t) via g quadruplex binding model

# 5. plot dc_/dt/A_GUV vs c_0-c_i (expect to see a positive linear relationship)


# as data includes actual G4 background for GUV permeability and later Gramicidin A experiments, start with these experiments


In [3]:
App = 3.16 #um^2

# G quadruplex model binding


def exp_I(c, k = 8.8,A = 0.39):
    return  A+ (1-A)/(k_1*c**2+1)

def inv(I, k = 8.8,A = 0.39):
    
    if I > 1:
        return 0
    
    elif I < A:
        return 1
        
    else:
        return np.sqrt((1/k)*((1-A)/(I-A)-1))


#smoothing configs

sigma = 30




# Calibration model fit


## **If you have already fitted a calibration function and saved the parameters skip this section**


In [4]:
# finding a suitable calibration

calib_data = pd.read_csv('../20210323_KCLFAMG4Q0to20mMCalib/RelIntensG41Concentration.csv')

In [5]:
raw_data = pd.read_csv('../20210323_KCLFAMG4Q0to20mMCalib/1uM/Intensities0-2mM.csv')
n = 512**2

print(raw_data['StdDev']/np.sqrt(n))

dI = raw_data['StdDev']/np.sqrt(n)
drelI = dI/raw_data['Mean']

drelI = drelI*10

rel_errorK = 0.02


0    2.244234
1    2.140793
2    1.921438
3    1.709535
4    1.638162
5    1.617246
Name: StdDev, dtype: float64


In [6]:
%matplotlib qt
I= np.linspace(1,0.2,100)

fig, ax = plt.subplots(1,1)

ax.scatter(calib_data['Rel. Intes'],calib_data['K conc [mM]'],marker = '+',s = 5)

legend = []

# fit a polynomial


fit = np.polyfit(-np.log(calib_data['Rel. Intes'].iloc[:-1]),calib_data['K conc [mM]'].iloc[:-1],3)

def fit_func(I,fit):
    
    c = np.zeros_like(I)
    
    for i in range(1,len(fit)+1):

        c += fit[i-1]*(-np.log(I))**(len(fit)-i)
        print(len(fit)-i)
    return c



ax.plot(I,fit_func(I,fit))

legend.append('Calibration')

def model_fit(cK,kb,c):
    return c + (1-c)/(kb*cK**2+1)

params = curve_fit(model_fit,calib_data['K conc [mM]'],calib_data['Rel. Intes'], p0 = [8.8,0.45])

cK_bar = np.linspace(0,2)
I_bar = model_fit(cK_bar,params[0][0],params[0][1])

ax.set_xlim([0.35,1.05])
ax.plot(I_bar,cK_bar,label = 'Model')
legend.append('Model')

l  = plt.errorbar(calib_data['Rel. Intes'],calib_data['K conc [mM]'],rel_errorK*calib_data['K conc [mM]'],drelI,c = 'k')

ax.set_xticks([0.4,0.6,0.8,1])
ax.set_xticklabels([0.4,0.6,0.8,1],fontsize  = 15)

ax.set_yticks([0,1,2])
ax.set_yticklabels([0,1,2],fontsize = 15)
ax.set_ylim([-0.1,2.1])
plt.tick_params(direction = 'in',top = True,bottom = True,right = True, left = True,length = 5)
ax.set_xlabel('Rel Intensity [a.u]',fontsize = 18)
ax.set_ylabel('[K]/[mM]',fontsize = 18)
l[0].remove()
plt.legend(legend)



3
2
1
0


In [4]:
fit_dict = {'Fitting Params':fit}
df_fit = pd.DataFrame(fit_dict)

df_fit.to_csv('../Calibrations/Calibration_approx_fit_params.csv')

NameError: name 'fit' is not defined

# Load calibration function parameters and define function


In [4]:
fitsdf = pd.read_csv('../Calibrations/Calibration_approx_fit_params.csv')

fit = fitsdf['Fitting Params']

def fit_func(I,fit):
    
    c = np.zeros_like(I)
    
    for i in range(1,len(fit)+1):

        c += fit[i-1]*(-np.log(I))**(len(fit)-i)
        print(len(fit)-i)
    return c

In [5]:

t0 = 100
t1 = 250
dt = 5/60

In [8]:
# load 260pM Gramicidin A experiment

dfI = pd.read_csv('./collatedIabs20210421OmpF.csv')
dfA = pd.read_csv('./collatedAabs20210421OmpF.csv')

In [9]:
dfI = dfI.iloc[:,1:]
dfA = dfA.iloc[:,1:]

In [10]:
dfA = dfA.fillna(0)
dfI = dfI.fillna(0)

# Remove small detected objects, most likely not GUVs

**check that the size is appropriate for the experiment**

In [11]:
# remove traces for GUVs below a certain area

App = 3.16 #um^2
Amin= 75*App*1e-12 #m^2

for col in dfI.columns:
    
    if col in dfA.columns:
        area = 1e-12*App*np.mean(dfA[col].iloc[:30])
        
        if area < Amin:
            dfI.pop(col)
            dfA.pop(col)

# Remove intensity curves with sudden drops
 These are likely from GUVs suddenly escaping the trap

In [12]:

def remove_burst_events(data,threshold = 0.3):
    
    for column in data.columns:
        
        if len(data[column][data[column] < threshold]) >=1:
            data.pop(column)
            

In [13]:
%matplotlib qt
fig, ax = plt.subplots(1,1)

for col in dfI.columns:
    ax.plot(dfI[col])

In [14]:
# normalise intensities

for col in dfI.columns:
    dfI[col] = dfI[col]/np.mean(dfI[col][:30])
remove_burst_events(dfI)

# Smooth data

## Smoothing strategy

1. In experiments where there was relatively fast decay in intensity initially, but slows down (as exponential decays do..), yet there is still a lot of noise, it may be necessary to smooth different regions with different 'cutoff's.
When the intensity has dropped, the signal to noise drops. What's more during the slower period of intensity decay, there is increasing likelihood that some long timescale event will have happened to distort the data. This long timescale noise may be of a similar timescale to the 'signal' in the early period of rapid decay. 

2. The **solution** chosen here, is to have the user select a timepoint to bisect the time traces. The early period up until that timepoint is smoothed with the 1st user input smoothing parameter (usually between 11-21). Then the later period of slow decay is smoother with much larger smoothing settings (50-80). In order to ensure a smooth 'stitching' of these differently smoothed traces, we concatenate and repeat the smoothing with the smallest of the two specified smoothing parameters.

In [15]:
def smooth_decay(I,t0,t1=-1,smoothing_0 = 17,smoothing_1 = 81):
    
    
    if smoothing_0 > t1-t0 and t1 != -1:
        raise ValueError('Smoothing parameter cannot be greater than input data')
    
    smoothed_decay_0 = I.iloc[t0:t1]
    
    smoothed_decay_0 = savgol_filter(smoothed_decay_0,smoothing_0,1)
    
    if t1 != -1:
        
        smoothed_decay_1 = I.iloc[t1:]
        smoothed_decay_1  = savgol_filter(smoothed_decay_1,smoothing_1,1)
        
        smoothed_decay = np.concatenate((smoothed_decay_0,smoothed_decay_1))
    else:
        smoothed_decay = smoothed_decay_0
                               
    final_smoothed_decay = savgol_filter(smoothed_decay,smoothing_0,1)
    return final_smoothed_decay
                                  
                                    

In [17]:
#Check smoothing still fits data
#
num = 8
fig,axs = plt.subplots(1,num)

for i,col in enumerate(dfI.columns[np.random.randint(0,len(dfI.columns),num)]):
    
    axs[i].plot(dfI[col].iloc[t0:].to_numpy())
    axs[i].plot(smooth_decay(dfI[col],t0,t1))
    

# Use calibration inverse to convert I to [K]

In [60]:
dfC_dict = {}

# can't exploit numpy broadcasting

for col in dfI.columns:
    
    smoothed_decay = smooth_decay(dfI[col],t0,t1)
#     print(smoothed_decay)
#     normalised_sd = smoothed_decay/np.mean(smoothed_decay[:30])
    
    cs = []
    for I in smoothed_decay:
#         print(I)
        cs.append(fit_func(I,fit))
    
    
    dfC_dict[col] = cs
    
dfC = pd.DataFrame(dfC_dict)

3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3


3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


...

3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2


2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3


3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1


In [48]:
dfC

,04001,04002,04003,04005,04006,03001,03002,03003,03004,03005,...,00003,00004,00005,00006,00007,00008,00009,00011,00012,00013
0,0.01115461092638734,-0.21208269972485805,-0.08455732779614283,0.07866321062844303,0.00465877945439857,0.07324401652587952,0.0005374875002743642,0.11022060462186255,0.004157946233595721,0.10392179797208657,...,0.010173585390276914,0.11058329173815508,0.011618222072911237,-0.06074111341252424,-0.06000863580907391,0.061433578038863484,-0.03547807031965912,0.04597613527253343,0.08207167774703274,0.00995697727869133
1,0.011300298749665882,-0.21022252769983013,-0.08563410911363625,0.07670994049332888,0.005826185013480439,0.0728465063777786,0.002778551021839477,0.11525760808022426,0.0039584878589772445,0.11179991374634146,...,0.009974923225563138,0.11719540812250429,0.015925243250041896,-0.06088575406345503,-0.05834741960331539,0.06331828266011753,-0.03554404914448121,0.04676635534705331,0.08266300506897675,0.012246579332779992
2,0.011445954652111717,-0.20836554136516994,-0.08671227882529157,0.07474990648632181,0.0069915713995089045,0.07244871945639321,0.005012240356171606,0.12024612971132863,0.0037589706913800864,0.11956093266937032,...,0.009776201868317506,0.12372389675112938,0.02020422394254643,-0.06103042119263979,-0.056689708646936024,0.06519693179224562,-0.03561003381573434,0.04755555138866062,0.08325370642865078,0.014528304453746938
3,0.011591578636276214,-0.20651174120400945,-0.08779183714127385,0.07278309377170242,0.008154939831994282,0.07205065564272702,0.007238563793500032,0.12518660075801688,0.0035593947248995323,0.12720643390164932,...,0.009577421312194502,0.13016976076434786,0.024455235107924717,-0.06117511480090615,-0.05503550161615936,0.06706953663910906,-0.03567602433353735,0.04834372405785029,0.08384378227343586,0.016802162727471343
4,0.011737170704714223,-0.20466112768599093,-0.08887278426766537,0.07080948774823091,0.009316291544010976,0.07165231481816797,0.009457529802813607,0.1300794695168482,0.003359759953642012,0.1347380945766452,...,0.009378581550859931,0.13653405538734026,0.02867835051179225,-0.06131983488908131,-0.053384797157514964,0.0689361085788128,-0.03574202069801051,0.04913087401955757,0.08443323305283934,0.01906816445485955
5,0.011882730859984075,-0.2028137012671729,-0.08995512040648317,0.06882907404684928,0.010475627782267821,0.07125369686448818,0.011669147033654324,0.13492520180984174,0.0031600663717257817,0.1421576940328651,...,0.009179682577991441,0.14281788983561733,0.03287364678321854,-0.061464581457990365,-0.05173759388763344,0.07079665916532502,-0.03580802290927248,0.04991700194317519,0.08502205921850069,0.02132632015407952
6,0.012028259104647578,-0.20096946238994012,-0.0910388457556966,0.0668418385284063,0.01163294980717982,0.0708548016638434,0.013873424317925896,0.13972428146977028,0.0029603139732798656,0.14946711823600528,...,0.008980724387279045,0.14902242929303794,0.037041203471152494,-0.06160935450845717,-0.050093890393040026,0.07265120013011042,-0.03587403096744335,0.050702108502570876,0.08561026122419756,0.023576640562816616
7,0.01217375544127004,-0.19912841148290972,-0.09212396050924473,0.0648477672814053,0.012788258892939233,0.07045562909877147,0.01607037067172317,0.14447721083943404,0.002760502752444741,0.15666836440062273,...,0.008781706972424048,0.1551488969654245,0.04118110310196347,-0.06175415404130275,-0.04845368522995039,0.07449974338377988,-0.03594004487264159,0.05148619437610308,0.08619783952585301,0.025819136640553828
8,0.01231921987242024,-0.1972905489608416,-0.09321046485705461,0.06284684661977381,0.013941556327587923,0.07005617905219136,0.018259995297176437,0.14918451128535903,0.0025606327033710933,0.16376354582126723,...,0.008582630327138518,0.16119857621296252,0.04529343123811577,-0.061898980057348785,-0.046816976924061816,0.07634230101775494,-0.03600606462498746,0.052269260246638104,0.08678479458154077,0.02805381957087426
9,0.012498005891076056,-0.19533053794289112,-0.09442582316906423,0.06144607805251941,0.015151011653048,0.0699606735096842,0

In [19]:
# filter concentration data by gradient
# if 

fig, axs = plt.subplots(1,2)

for col in dfC.columns:
    
    grad = np.gradient(dfC[col])
    
#     if len(grad[grad <= -0.0009])/len(grad) > 0.05:
#         dfC.pop(col)
#         continue
    axs[0].hist(grad,15)
    axs[1].plot(dfC[col])


In [56]:
tifs = glob('/media/marcus/9C33-6BBD/20210421_OmpF_PCPG_KCl_Exp_1/20210421_OmpF_PCPG_KCl_Exp_1_MMStack_Pos0.ome.tif')
tif = tifs[-1]

vid = tf.TiffFile(tif)

frames = vid.asarray()
bg = np.median(frames,axis = (1,2))
bg = bg[90:]
bg = np.array(bg)-np.mean(bg[-30:-20])
bg = bg/np.max(bg)

smoothed_bg = medfilt(bg,11)

OME series: invalid TiffData index
OME series: invalid TiffData index


In [57]:
approx_k_conc = 1-smoothed_bg

In [61]:
%matplotlib qt
fig, ax = plt.subplots(1,1)

for col in dfI.columns:
    ax.plot(np.arange(len(dfI[col].iloc[100:])),dfI[col].iloc[100:])
    
    
t_0_record = 100
ax.plot(np.arange(len(approx_k_conc[t_0_record:])),approx_k_conc[t_0_record:])

In [13]:
# find representative background concentration variation

# tifs = glob('/media/marcus/9C33-6BBD/20210920_GramA_260pM07_3nM815_1mMKCl_1/260pM/*.tif')

# bgs = []

# fig, ax = plt.subplots(1,1)

# for tif in tifs:
    
#     vid = tf.TiffFile(tif)
    
#     frames = vid.asarray()
    
#     bg = np.median(frames,axis  =(1,2))
#     bg = np.array(bg)/np.max(bg)
#     bgs.append(bg)
    
    
#     ax.plot(bg)

# This confirms that the normalised background is the same for each chamber. Suggesting potassium concs are identical between chambers



In [131]:
# c_bg = []
# for x in bg[360+t0:]:
#     c_bg.append(inv(x))

In [132]:
# fig, axs = plt.subplots(1,2)


# smoothed_background = gaussian(c_bg,sigma)
# axs[0].plot(smoothed_background)
# axs[1].plot(bg[360+t0:])

# Find concentration difference

In [68]:
# calculating c_ext - c_i

dfdeltaC_dict = {}

for col in dfC.columns:
    deltaC = []
    for i in range(len(dfC[col])):
    
        deltaC.append(approx_k_conc[t_0_record:][i]-dfC[col].iloc[i])
    dfdeltaC_dict[col] = deltaC
dfdeltaC = pd.DataFrame(dfdeltaC_dict)# drift in the background intensity over time, once the potassium has arrived, is causing a drifting predicted potassium concentration
# this is unlikely to be the case in reality.
# Therefore the best thing may be to just assume that after an initial 80 frame boundary region (20 mins) for this exp
# the potassium concentration externally reaches 1mM.


In [70]:
dfdeltaC_dict.keys()


dict_keys(['04001', '04002', '04003', '04005', '04006', '03001', '03002', '03003', '03004', '03005', '03006', '03007', '03008', '03009', '03010', '03011', '03012', '03014', '03016', '03017', '03018', '02001', '02002', '02003', '02004', '02005', '02006', '02007', '02008', '02009', '02011', '02012', '02013', '01001', '01002', '01004', '01005', '01006', '01007', '01008', '01009', '01011', '01012', '01014', '01015', '01016', '01017', '01018', '01019', '01020', '01021', '01022', '01023', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00011', '00012', '00013'])

In [47]:
len(dfC[col]),len(approx_k_conc[t_0_record:])

(673, 764)

In [71]:
fig, axC = plt.subplots(1,1)

delete_cols = []

for col in dfdeltaC.columns:
    graddeltaC = np.gradient(dfdeltaC[col])
    
    
#     if len(graddeltaC[graddeltaC > 0]) > 10:
#         delete_cols.append(col)
#         continue
        
    axC.plot(dfdeltaC[col])

In [72]:
fig, axC = plt.subplots(1,1)

delete_cols = []

for col in dfdeltaC.columns:
    graddeltaC = np.gradient(dfdeltaC[col])
    
    
#     if len(graddeltaC[graddeltaC > 0]) > 1:
#         delete_cols.append(col)
#         continue
        
    axC.plot(dfdeltaC[col])

In [73]:
for col in delete_cols:
    dfdeltaC.pop(col)

# Determine the flux

In [74]:
# now calculate J

# J = (1/A)*dN/dt = R/3* dc_i/dt

dfJ_dict = {}


for col in dfdeltaC:
    
    if col in dfA:
        
        delta_c = np.gradient(dfC[col],dt)/60 #molm^-3s^-1
        
        A = np.mean(dfA[col].iloc[:30])*App*1e-12 #m^2
        
        R = np.sqrt(A/np.pi) #m
        
        dfJ_dict[col] = R/3*delta_c #molm^-2s^-1
        

dfJ = pd.DataFrame(dfJ_dict)



In [76]:
fig, ax = plt.subplots(1,1)

for col in dfJ.columns:
    ax.plot(dfJ[col])

In [79]:
fig, axs = plt.subplots(1,3)
legend = []

for col in dfJ.columns:

    legend.append(col)
#     print(dfdeltaC)

    axs[0].scatter(dfdeltaC[col],dfJ[col]*1e10, marker = '+', s = 10)
    print(dfJ[col])

    axs[1].plot(dt*np.arange(0,len(dfdeltaC[col])),dfdeltaC[col])

    
#     gradients = []
#     win = 10
#     for i in range(dfJ[col].shape[0]-(win-1)):
        
#         gradient = np.mean(np.gradient(dfJ[col].iloc[i:i+win]))
#         gradients.append(gradient)
        
    
    axs[2].plot(np.gradient(dfJ[col],dfdeltaC[col]))
axs[0].tick_params(direction = 'in',left = True,right = True,bottom = True,top = True,length = 6)
axs[1].tick_params(direction = 'in',left = True,right = True,bottom = True,top = True,length = 6)
axs[0].set_xlabel('[K_ext]-[K_int] [mM]',fontsize = 18)

axs[0].set_ylabel('J [molm^-2s^-1]',fontsize = 18)

axs[1].set_ylabel('[K_int] [mM]',fontsize = 18)


axs[1].set_xlabel(' T [mins]',fontsize = 18)



#xticks

axs[0].set_xticks([0.5,0.75,1])
axs[0].set_xticklabels([0.5,0.75,1],fontsize = 15)

axs[1].set_xticks([0,20,40])
axs[1].set_xticklabels([0,20,40],fontsize = 15)

#yticks

axs[0].set_yticks([0,0.005,0.01,0.015])
axs[0].set_yticklabels([0,0.005,0.01,0.015],fontsize = 15)

axs[0].set_ylim([-0.0005,0.0155])
axs[1].set_yticks([0, 0.25,0.5])
axs[1].set_yticklabels([0,0.25,0.5],fontsize = 15)


# plt.rcParams['text.usetex'] = True

# def myticks(x,pos):

#     if x == 0: return "$0$"

#     exponent = int(np.log10(x))
#     coeff = x/10**exponent

#     return r"${:2.0f} \times 10^{{ {:2d} }}$".format(coeff,exponent)

# axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(myticks))

0      1.335491e-10
1      1.335345e-10
2      1.335052e-10
3      1.334760e-10
4      1.334467e-10
5      1.334175e-10
6      1.333882e-10
7      1.333590e-10
8      1.486170e-10
9      2.019091e-10
10     2.254728e-10
11     2.359556e-10
12     3.225050e-10
13     3.899816e-10
14     4.050042e-10
15     4.671642e-10
16     5.006809e-10
17     5.154964e-10
18     5.682043e-10
19     5.162091e-10
20     3.240701e-10
21     2.460612e-10
22     3.229999e-10
23     4.662740e-10
24     5.023994e-10
25     3.694937e-10
26     2.412892e-10
27     1.770229e-10
28     1.559399e-10
29    -1.784628e-11
           ...     
643   -1.089356e-10
644   -1.089532e-10
645   -1.089708e-10
646   -1.089884e-10
647   -1.090060e-10
648   -1.090236e-10
649   -1.090412e-10
650   -1.090588e-10
651   -1.090764e-10
652   -1.090940e-10
653   -1.091116e-10
654   -1.091292e-10
655   -1.091468e-10
656   -1.091644e-10
657   -1.091821e-10
658   -1.091997e-10
659   -1.092173e-10
660   -1.092349e-10
661   -1.092525e-10


/home/marcus/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1069: RuntimeWarning: divide by zero encountered in true_divide
  a = -(dx2)/(dx1 * (dx1 + dx2))
/home/marcus/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1071: RuntimeWarning: divide by zero encountered in true_divide
  c = dx1 / (dx2 * (dx1 + dx2))


0      1.160257e-09
1      1.158988e-09
2      1.156452e-09
3      1.153918e-09
4      1.151388e-09
5      1.148860e-09
6      1.146334e-09
7      1.143812e-09
8      1.124959e-09
9      1.163488e-09
10     1.218789e-09
11     1.233206e-09
12     1.251856e-09
13     1.211848e-09
14     1.162585e-09
15     1.214375e-09
16     1.265788e-09
17     1.208906e-09
18     1.177116e-09
19     1.105526e-09
20     1.060079e-09
21     1.046213e-09
22     9.696088e-10
23     8.875340e-10
24     8.461404e-10
25     9.145835e-10
26     8.956313e-10
27     8.830232e-10
28     9.851678e-10
29     1.103174e-09
           ...     
643    4.144850e-10
644    4.148913e-10
645    4.152980e-10
646    4.157051e-10
647    4.161127e-10
648    4.165207e-10
649    4.169291e-10
650    4.173380e-10
651    4.177473e-10
652    4.181570e-10
653    4.185672e-10
654    4.189779e-10
655    4.193890e-10
656    4.198005e-10
657    4.202124e-10
658    4.206248e-10
659    4.210377e-10
660    4.214510e-10
661    4.218647e-10


[Text(0, 0, '0'), Text(0, 0, '0.25'), Text(0, 0, '0.5')]

# Plot concentration difference


In [88]:
fig, ax = plt.subplots(1,1)

for col in dfJ.columns:

    ax.plot(dt*np.arange(27,len(dfdeltaC[col])),dfdeltaC[col].iloc[27:].to_numpy())

# Plot the Flux against concentration gradient



In [89]:
%matplotlib qt
figJ, axJ = plt.subplots(1,1)
figC, axC = plt.subplots(1,1)



def Jfit(deltac,k,c):
    return k*deltac + c

p0 = [1e-14,0]
Pparams = {}
cols = []
for i,col in enumerate(dfJ.columns):
    
    
    t = dt*np.arange(0,len(dfC[col]))
    
    J = dfJ[col]
    deltaC = dfdeltaC[col]
    
#     deltaC_bar = np.linspace(dfC[col].min(),1)
    
    params = curve_fit(Jfit,deltaC.iloc[deltaC.to_numpy().astype(float)>0.7],J.iloc[deltaC.to_numpy().astype(float)>0.7],p0)
    
    J_bar = Jfit(deltaC.iloc[deltaC.to_numpy().astype(float)>0.7],*params[0])
    P = params[0][0]
    dP = np.sqrt(params[1][0][0])
    print(dP/P, P)
    
    Pparams[col] = params
    
    if P < 0:
        
        continue
        
    if len(J.iloc[(deltaC < 0.7).to_numpy()]) < 1:
        continue
    
    cols.append(col)
    
    print(col)
    # plot fluxes
    axJ.scatter(deltaC,J,alpha = 0.25)
#     axJ.plot(deltaC.iloc[deltaC.to_numpy().astype(float)>0.85],J_bar)
    
    #plot the Intensity vs time
    
    axC.plot(t,dfC[col])
 
#Flux plot params


axJ.set_xticks([0,0.5,1])
axJ.set_xlim([-0.05,1.05])
axJ.set_xticklabels([0,0.5,1],fontsize = 15)

axJ.set_xlabel('$[K]_{ext} - [K]_{in}$ [mM]',fontsize = 18)
axJ.set_yticks([0,5e-9,1e-8,1.5e-8])
axJ.set_yticklabels([0,5,10,15],fontsize = 15)

axJ.set_ylim([-1e-9,16e-9])
#  
axJ.set_ylabel('J  [$10^{-12}$ mol$m^{-2}s^{-1}$]',fontsize = 18)
axJ.tick_params(direction = 'in',left = True,right = True, top = True, bottom = True,length = 7)

axC.set_xticks([0,20,40])
axC.set_xticklabels([0,20,40],fontsize = 15)
axC.set_xlabel('Time [mins]',fontsize = 18)

axC.set_yticks([0,0.5,1])
axC.set_yticklabels([0,0.5,1],fontsize = 15)
axC.set_ylabel('Rel I',fontsize = 18 )
axC.set_ylim([-0.05,1.05])
axC.set_xlim([-2,45])

# global params

axC.tick_params(direction = 'in',left = True,right = True, top = True, bottom = True,length = 7)


-0.1210851919899276 -1.0647418694865651e-09
0.042024452900606565 8.230849602321814e-09
04002
0.4749195371521377 1.0580371460943217e-09
04003
0.3213343335113911 1.4429162893356364e-08
04005
-0.21245525485229066 -8.471648421277725e-10
0.27264961225920786 1.3111577292935382e-09
03001
0.07549281214142911 6.747069199290695e-09
03002


TypeError: Improper input: N=2 must not exceed M=0

# Save permeability values

In [37]:
P_df = {'P ms^-1': [Pparams[key][0][0] for key in Pparams.keys()],
       'dP/P': [np.sqrt(Pparams[key][1][0][0])/Pparams[key][0][0] for key in Pparams.keys()],
       'A':[np.mean(dfA[key].iloc[:30]) for key in Pparams.keys()]}
        
P_identified = {key:[Pparams[key][0][0]] for key in Pparams.keys()}
P_identified = pd.DataFrame(P_identified)
P_df = pd.DataFrame(P_df)

In [38]:
P_df['P cms^-1'] = P_df['P ms^-1']*100

In [39]:
P_df.to_csv('Permeabilities_OmpF_20210427.csv')

# Save Flux vs deltaC data

In [31]:
dfJ.to_csv('Flux_OmpF_20210427.csv')
dfdeltaC.to_csv('deltaC_OmpF_20210427.csv')
P_identified.to_csv('PbyGUVId_OmpF_20210427.csv')

# Mean J vs Mean concentration difference

Need to bin the concentration differences and then average the flux within each bin


In [32]:
deltaCbins = np.linspace(0.35,1,100)

deltaC_arr = dfdeltaC[cols].to_numpy()

dfJoverP = dfJ[cols]

for col in dfJoverP.columns:
    dfJoverP[col] = dfJoverP[col].to_numpy()/P_identified[col].iloc[0]
    
    
J_arr = dfJoverP.to_numpy()

J_mean = []
deltaC_mean = []

for i in range(len(deltaCbins)-1):
    
    deltaC_mean.append(np.median(deltaC_arr[(deltaC_arr < deltaCbins[i+1]) * (deltaC_arr >deltaCbins[i])]))
    J_mean.append(np.median(J_arr[(deltaC_arr < deltaCbins[i+1]) * (deltaC_arr >deltaCbins[i])]))


/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [33]:
figm,axm = plt.subplots(1,1)


axm.scatter(deltaC_mean,J_mean)
# axm.set_ylim([-1e-9,16e-9])

In [34]:
meanFluxdata = {'deltaC':deltaC_mean,'J': J_mean}

meanFluxdata = pd.DataFrame(meanFluxdata)

meanFluxdata.to_csv('20210420_OmpF_MeanFluxvsdeltaC.csv')